In [1]:
import warnings

warnings.simplefilter("ignore")

import torch

from som_based_rag import SOM_Based_RAG_Util

from wiki_datasource import WikiEventsDataSource

from openai_vector_encoder import OpenAIEmbeddingsVectorEncoder

from openai_qa_chatbot import OpenAIQuestionAnswerChatBot

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
openai_key = "sk-proj-cecR6SLBRlGmjFNWFkpPT3BlbkFJtR4vtIrgsuIxZB8yJJWc"

openai_vector_encoder_id = "text-embedding-ada-002"

openai_encoded_vector_dimensions = 1536

openai_tokenizer_name = "cl100k_base" 

openai_model_name = "gpt-3.5-turbo-instruct"

vector_encoder = OpenAIEmbeddingsVectorEncoder( openai_encoded_vector_dimensions, openai_vector_encoder_id, openai_key )

In [4]:
event_years_to_fetch = [ 2022, 2023, 2024 ]
data_source = WikiEventsDataSource( event_years_to_fetch  )

In [5]:
som_driven_rag_util = SOM_Based_RAG_Util( 
                                            vector_encoder = vector_encoder,
                                            som_lattice_height = 20,
                                            som_lattice_width = 30,
                                            learning_rate = 0.3,
                                            topk_bmu_for_indexing = 10,
                                            device = device
                                        )

In [6]:
som_driven_rag_util.load_n_vectorize_data( data_source )

Vectorized Data Batch:   0%|          | 0/5 [00:00<?, ?it/s]

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
som_driven_rag_util.vectors.shape

In [ ]:
som_driven_rag_util.train_n_index_data_vectors( train_epochs = 100 )

In [ ]:
som_driven_rag_util.find_semantically_similar_data( "Who aquired Twitter in 2022?", sim_threshold = 0.8 )

In [ ]:
openai_chatbot = OpenAIQuestionAnswerChatBot( 
                                                vector_db_util = som_driven_rag_util,
                                                openai_tokenizer_name = openai_tokenizer_name,
                                                openai_model_name = openai_model_name,
                                                openai_key = openai_key,
                                                question_input_max_token_count = 100,
                                                context_trim_percent = 0.1,
                                                device = device
                                            )

In [ ]:
sample_questions = [
                        "Who won the 2022 soccer world cup?",
                        "When did Sweden join NATO?",
                        "Who joined NATO in 2023?",
                        "Who joined NATO in 2024?",
                        "Which is the 31st member of NATO?",
                        "Which is the 32nd member of NATO?",
                        "Who won the Cricket World Cup in 2023?",
                        "Who defeated India in Cricket World Cup final in 2023?",
                        "Name the former prime minister of Japan that was assassinated in 2022?",
                        "When did Chandrayaan-3 land near the south pole of the Moon?",
                        "Where did Chandrayaan-3 land on the Moon?",
                        "Who acquired Twitter in 2022?",
                        "Who owns Twitter?",
                        "Who acquired Activision Blizzard in 2023?"
                   ]

In [ ]:
for sample_question in sample_questions:
    print( f"\nQuestion: {sample_question}" )
    answer = openai_chatbot.find_answer_to_question_without_context( sample_question )
    print( f"Answer: {answer}" )

In [ ]:
sim_threshold = 0.75

for sample_question in sample_questions:
    print( f"\nQuestion: {sample_question}" )
    answer = openai_chatbot.find_answer_to_question( sample_question, sim_threshold = sim_threshold)
    print( f"Answer: {answer}" )